The SPIMI (Single-Pass In-Memory Indexing) algorithm is a technique used for constructing a positional index. 


First, we will define methods for preprocessing the documents: tokenization, stop word removal, and stemming. Then we'll implement the SPIMI algorithm to construct the positional index.

Utility Functions for Preprocessing
Let's start by implementing the preprocessing steps:




In [16]:
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import defaultdict

# Load stop words
stop_words = set(stopwords.words('english'))

# Initialize the Porter Stemmer
ps = PorterStemmer()

def preprocess(text):
    # Tokenization
    tokens = re.findall(r'\b\w+\b', text.lower())
    # Stop word removal and stemming
    preprocessed_tokens = [ps.stem(token) for token in tokens if token not in stop_words]
    return preprocessed_tokens



In [ ]:
SPIMI Algorithm for Index Construction
Now let's implement the SPIMI algorithm to construct the positional index:

In [18]:
def spimi_invert(documents):
    """
    SPIMI algorithm to construct a positional index.
    
    Parameters:
    documents (list of str): List of documents to index
    
    Returns:
    dict: Positional index
    """
    # Initialize the positional index
    index = defaultdict(lambda: defaultdict(list))
    
    for doc_id, document in enumerate(documents):
        tokens = preprocess(document)
        for position, token in enumerate(tokens):
            index[token][doc_id].append(position)
    
    return index

# Example usage
documents = [
    "The quick brown fox jumps over the lazy dog.",
    "Never jump over the lazy dog quickly.",
    "A quick brown dog outpaces a quick fox."
]

positional_index = spimi_invert(documents)
print(positional_index)
# Print the positional index
for term, postings in positional_index.items():
    print(f"Term: {term}")
    for doc_id, positions in postings.items():
        print(f"  Document ID: {doc_id}, Positions: {positions}")


defaultdict(<function spimi_invert.<locals>.<lambda> at 0x00000166F3725440>, {'quick': defaultdict(<class 'list'>, {0: [0], 2: [0, 4]}), 'brown': defaultdict(<class 'list'>, {0: [1], 2: [1]}), 'fox': defaultdict(<class 'list'>, {0: [2], 2: [5]}), 'jump': defaultdict(<class 'list'>, {0: [3], 1: [1]}), 'lazi': defaultdict(<class 'list'>, {0: [4], 1: [2]}), 'dog': defaultdict(<class 'list'>, {0: [5], 1: [3], 2: [2]}), 'never': defaultdict(<class 'list'>, {1: [0]}), 'quickli': defaultdict(<class 'list'>, {1: [4]}), 'outpac': defaultdict(<class 'list'>, {2: [3]})})
Term: quick
  Document ID: 0, Positions: [0]
  Document ID: 2, Positions: [0, 4]
Term: brown
  Document ID: 0, Positions: [1]
  Document ID: 2, Positions: [1]
Term: fox
  Document ID: 0, Positions: [2]
  Document ID: 2, Positions: [5]
Term: jump
  Document ID: 0, Positions: [3]
  Document ID: 1, Positions: [1]
Term: lazi
  Document ID: 0, Positions: [4]
  Document ID: 1, Positions: [2]
Term: dog
  Document ID: 0, Positions: [5]
 

Explanation
Preprocessing: The preprocess function tokenizes the input text, removes stop words, and applies stemming.
SPIMI Algorithm: The spimi_invert function iterates through each document, preprocesses it, and then updates the positional index with the term positions for each document.
This approach ensures that each term in the positional index maps to a dictionary, where the keys are document IDs and the values are lists of positions of the term within the document.

By running the example usage, you can see how the positional index is constructed for the given documents.

In [ ]:
Optimizing Boolean query evaluation using the inverted index involves leveraging the structure of the index to quickly retrieve and intersect postings lists. Here’s how you can implement and optimize Boolean query evaluation:

In [ ]:
Step-by-Step Approach
Convert the Query to Conjunctive Normal Form (CNF): This allows us to handle queries involving AND, OR, and NOT.
Retrieve Postings Lists: For each term in the query, retrieve its postings list from the index.
Optimize Intersection Order: Intersect postings lists starting with the smallest to minimize the number of comparisons.
Perform the Boolean Operations: Intersect (AND), union (OR), and difference (NOT) on postings lists.

In [ ]:
Implementation
Let’s start by implementing utility functions for Boolean operations on postings lists:

In [20]:
def intersect(postings1, postings2):
    """Intersect two postings lists."""
    i, j = 0, 0
    result = []
    while i < len(postings1) and j < len(postings2):
        if postings1[i] == postings2[j]:
            result.append(postings1[i])
            i += 1
            j += 1
        elif postings1[i] < postings2[j]:
            i += 1
        else:
            j += 1
    return result

def union(postings1, postings2):
    """Union two postings lists."""
    i, j = 0, 0
    result = []
    while i < len(postings1) and j < len(postings2):
        if postings1[i] == postings2[j]:
            result.append(postings1[i])
            i += 1
            j += 1
        elif postings1[i] < postings2[j]:
            result.append(postings1[i])
            i += 1
        else:
            result.append(postings2[j])
            j += 1
    result.extend(postings1[i:])
    result.extend(postings2[j:])
    return result

def difference(postings1, postings2):
    """Difference between two postings lists."""
    i, j = 0, 0
    result = []
    while i < len(postings1) and j < len(postings2):
        if postings1[i] == postings2[j]:
            i += 1
            j += 1
        elif postings1[i] < postings2[j]:
            result.append(postings1[i])
            i += 1
        else:
            j += 1
    result.extend(postings1[i:])
    return result


In [ ]:
Boolean Query Evaluation
Let's implement a function to evaluate Boolean queries using the inverted index:

In [30]:
def evaluate_boolean_query(query, index):
    """
    Evaluate a Boolean query on the positional index.
    
    Parameters:
    query (str): Boolean query in CNF
    index (dict): Positional index
    
    Returns:
    list: List of document IDs satisfying the query
    """
    # Split the query into terms and operators
    terms = re.findall(r'\b\w+\b', query.lower())
    operators = re.findall(r'AND|OR|NOT', query.upper())

    # Preprocess terms
    terms = [ps.stem(term) for term in terms if term not in stop_words]
    print (f"terms: {terms}")
    

    # Retrieve postings lists for each term
    postings_lists = {term: sorted(index.get(term, {}).keys()) for term in terms}
    print (f"postings_lists: {postings_lists}")
    print (type(postings_lists))

    # Evaluate the query
    if not operators:
        print ('1 here')
        return postings_lists.get(terms[0], [])

    result = postings_lists[terms[0]]
    i = 1
    print (f"result:{result}")

    while i < len(terms):
        operator = operators[i - 1]
        if operator == 'AND':
            print ('2 here')
            result = intersect(result, postings_lists[terms[i]])
            print (f"parameter :{postings_lists[terms[i]]}")
        elif operator == 'OR':
            result = union(result, postings_lists[terms[i]])
        elif operator == 'NOT':
            result = difference(result, postings_lists[terms[i]])
        i += 1

    print ('3 here')
    return result

# Example usage
query = "quick AND dog"
result = evaluate_boolean_query(query, positional_index)

print(f"Documents matching the query '{query}': {result}")


terms: ['quick', 'dog']
postings_lists: {'quick': [0, 2], 'dog': [0, 1, 2]}
<class 'dict'>
result:[0, 2]
2 here
parameter :[0, 1, 2]
3 here
Documents matching the query 'quick AND dog': [0, 2]


In [ ]:
xplanation
Boolean Operations: We have defined functions for intersect, union, and difference operations on postings lists.
Query Evaluation: The evaluate_boolean_query function parses the query, retrieves postings lists for each term, and evaluates the query using the Boolean operations.
Optimization: By starting with the smallest postings list for intersections, we minimize the number of comparisons, which improves efficiency.

In [ ]:
Example Usage
Running the provided code with the example documents and query will output the document IDs that match the Boolean query "quick AND dog".

By following this approach, you can efficiently evaluate Boolean queries using the inverted index and optimize the process to handle large document collections effectively.